# Model Training

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch import optim
from tqdm import tqdm
import os

from code.train_model import ModelTrainer
from code.function import load_train_data, load_test_data

from model.gru_base import GRUModel
from model.gru_patch import GRUPatchModel

## load data

load the processed data and split the train set, test set and valid set

In [2]:
X = np.load('data/processed_data/X.npy')
ret5 = np.load('data/processed_data/ret5.npy')
ret10 = np.load('data/processed_data/ret10.npy')

sample_datetime = np.load('data/processed_data/sample_datetime.npy', allow_pickle=True)
sample_stock = np.load('data/processed_data/sample_stock.npy', allow_pickle=True)
print(X.shape, ret5.shape, ret10.shape, sample_datetime.shape, sample_stock.shape)

(5272, 2498, 6) (5272, 2498) (5272, 2498) (2498,) (5272,)


In [3]:
train_end = int(len(sample_datetime) * 0.9)
valid_end = int(len(sample_datetime))

seq_len = 40

X_train, X_valid = X[:, :train_end], X[:, train_end-seq_len:valid_end]
y_train, y_valid = ret10[:, :train_end], ret10[:, train_end-seq_len:valid_end]
train_date, valid_date = sample_datetime[:train_end], sample_datetime[train_end:valid_end]

print(X_train.shape, y_train.shape, train_date.shape)
print(X_valid.shape, y_valid.shape, valid_date.shape)

(5272, 2248, 6) (5272, 2248) (2248,)
(5272, 290, 6) (5272, 290) (250,)


In [4]:
x1_train, x1_test, y_train, y_test = load_train_data(X_train, y_train)
BATCH_SIZE = 5000

class Newdataset(Dataset):
    def __init__(self, data1, label) -> None:
        super().__init__()
        self.data1 = data1.astype(np.float32)
        self.label = label.astype(np.float32)

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, index):
        return self.data1[index], self.label[index]

train_ds = Newdataset(x1_train, y_train)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

test_ds = Newdataset(x1_test, y_test)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

## set model

set up the model trainer

In [5]:
model1 = GRUModel()
model2 = GRUPatchModel()

optimizer1 = optim.Adam(model1.parameters(), lr=0.005)
optimizer2 = optim.Adam(model2.parameters(), lr=0.005)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

modeltrainer1 = ModelTrainer(model1, optimizer1, device, 'gru')
modeltrainer2 = ModelTrainer(model2, optimizer2, device, 'gru_patch')

if not os.path.exists(f'data/saved_model'):
    os.makedirs(f'data/saved_model')

train model

In [9]:
# modeltrainer1.fit(train_dl, test_dl, 'data')
# modeltrainer2.fit(train_dl, test_dl, 'data')

current device: cpu
begin time: Thu Apr 11 14:48:41 2024
GRUModel(
  (gru): GRU(6, 30, batch_first=True)
  (head): PredictionHead(
    (dropout): Dropout(p=0.0, inplace=False)
    (hidden): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (linear): Linear(in_features=30, out_features=1, bias=True)
  )
)
Set random seed as 0 for pytorch


epoch:21 test IC:0.0675 : 100%|██████████| 55/55 [00:06<00:00,  8.18it/s]


current device: cpu
begin time: Thu Apr 11 15:08:48 2024
GRUPatchModel(
  (gru): GRU(5, 30, batch_first=True)
  (flatten): Flatten_Head(
    (linears): ModuleList(
      (0-5): 6 x Linear(in_features=30, out_features=1, bias=True)
    )
    (dropouts): ModuleList(
      (0-5): 6 x Dropout(p=0.1, inplace=False)
    )
    (flattens): ModuleList(
      (0-5): 6 x Flatten(start_dim=-2, end_dim=-1)
    )
    (batch0): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batch1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (outlinear): Sequential(
      (0): ReLU()
      (1): Linear(in_features=6, out_features=1, bias=True)
    )
  )
)
Set random seed as 0 for pytorch


epoch:12 test IC:0.0478 : 100%|██████████| 55/55 [00:27<00:00,  2.03it/s]


([0.04387389672326071,
  0.06758110345283855,
  0.07035817737695357,
  0.0728832052331673,
  0.074260747553483,
  0.07505820604406943,
  0.07559273422155614,
  0.0772748186410358,
  0.07864979873560199,
  0.08026525121369318,
  0.08169258915347187,
  0.08476456377195986,
  0.08666225340744677],
 [0.052669897506182846,
  0.051933313652195716,
  0.05320246367429553,
  0.04742696102369915,
  0.05128775657239285,
  0.04402481748468497,
  0.05169810431755402,
  0.057701658908362416,
  0.05252456740814854,
  0.04641687252812765,
  0.050467050346461206,
  0.056259666696529494,
  0.04779086589008908])

## predict factor

use the model in valid set and get the factor output

In [11]:
# load model
model1 = torch.load('data/saved_model/gru.pt')
model2 = torch.load('data/saved_model/gru_patch.pt')

optimizer1 = optim.Adam(model1.parameters(), lr=0.005)
optimizer2 = optim.Adam(model2.parameters(), lr=0.005)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

modeltrainer1 = ModelTrainer(model1, optimizer1, device, 'gru')
modeltrainer2 = ModelTrainer(model2, optimizer2, device, 'gru_patch')

In [12]:
def predict_valid_set(X_valid, y_valid, modeltrainer, valid_date, sample_stock):
    fac_1 = pd.DataFrame(np.nan * np.zeros((X_valid.shape[0], len(valid_date)-10)))
    i_panel = 0
    for i in tqdm(range(len(valid_date)-10)):
        x1_test, y_test, nonan_index = load_test_data(X_valid[:, i:i+seq_len, :], y_valid[:, i:i+seq_len])
        test_ds = Newdataset(x1_test, y_test)
        test_dl = DataLoader(test_ds, batch_size=len(x1_test))

        y_pred = modeltrainer.predict(test_dl)
        fac_1.iloc[nonan_index, i_panel] = y_pred[:, -1]
        i_panel += 1
    fac_1.columns = valid_date[:i_panel]
    fac_1.index = sample_stock
    return fac_1

In [13]:
fac1 = predict_valid_set(X_valid, y_valid, modeltrainer1, valid_date, sample_stock)
fac2 = predict_valid_set(X_valid, y_valid, modeltrainer2, valid_date, sample_stock)

100%|██████████| 240/240 [02:21<00:00,  1.70it/s]


In [14]:
if not os.path.exists(f'data/saved_factor/'):
    os.makedirs(f'data/saved_factor/')

fac1.to_csv('data/saved_factor/fac1.csv')
fac2.to_csv('data/saved_factor/fac2.csv')